# Defining jobs in jobflow

In this tutorial, you will:

- Learn about the `job` decorator.
- Understand the structure of the `Job` object.
- Set the configuration settings of a job.
- Use the `Response` object.
- Learn tips for writing job functions.

The purpose of this tutorial is to delve into the basic functionality of jobs and gain a feeling for what is possible. Later tutorials will describe how to employ jobs in complex workflows.

## Creating job objects

The building block of jobflows are `Job` objects. Jobs are delayed calls to python functions whose outputs are stored in a database. The easiest way to create a job is using the `@job` decorator. The job decorator can be applied to any function, even those with optional parameters.


In [2]:
import warnings

warnings.filterwarnings("ignore", "Using `tqdm.autonotebook.tqdm`")

In [3]:
from jobflow import job


@job
def add(a, b, c=2):
    return a + b + c

Any call to the `add` function will return a `Job` object.


In [4]:
add_first = add(1, 2, c=5)

Each job is assigned a unique identifier (UUID).


In [5]:
add_first.uuid

'eff8821e-d105-4072-b13b-8a4dcb2fbfa5'

Jobs also have an index. This tracks the number of times the job has been "replaced" (replacing is covered in detail in the [Dynamic and nested Flows tutorial](dynamic-flows)).


In [6]:
add_first.index

1

Jobs have outputs that can be accessed using the `output` attribute. As the job has not yet been executed, the output is currently a reference to the future output.


In [7]:
add_first.output

OutputReference(eff8821e-d105-4072-b13b-8a4dcb2fbfa5)

The output of a job can be used as the input to another job.


In [8]:
add_second = add(add_first.output, 3)

The output does not have to be an argument on its own, it can be included in a list or a dictionary.


In [9]:
@job
def sum_numbers(numbers):
    return sum(numbers)


sum_job = sum_numbers([add_first.output, add_second.output])

# Running Jobs


Here, we will demonstrate how to run a simple job locally, which can be useful for testing purposes.


In [10]:
from jobflow.managers.local import run_locally

response = run_locally(add(1,2))

2023-06-07 20:24:55,628 INFO Started executing jobs locally
2023-06-07 20:24:56,021 INFO Starting job - add (da48e4d6-d6c2-46ad-9bec-4a44e30a10fe)
2023-06-07 20:24:56,022 INFO Finished job - add (da48e4d6-d6c2-46ad-9bec-4a44e30a10fe)
2023-06-07 20:24:56,023 INFO Finished executing jobs locally
{'da48e4d6-d6c2-46ad-9bec-4a44e30a10fe': {1: Response(output=5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}}


The output contains a UUID for the `job` along with its outputs.


In [11]:
print(response)

{'da48e4d6-d6c2-46ad-9bec-4a44e30a10fe': {1: Response(output=5, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False)}}
